<https://leemeng.tw/practical-pandas-tutorial-for-aspiring-data-scientists.html>
建立DataFrame
=
用python建立DataFrame
-
用dict初始化DataFrame很直覺，dict 內 key 值對應到欄位名稱，value 則是一個iterable，代表該欄位裏頭所有數值。

In [ ]:
import pandas as pd

dic = {
    'col 1': [1,2,3],
    'col 2': [10,20,30],
    'col 3': list('xyz'),
    'col 4': ['a','b','c'],
    'col 5': pd.Series(range(3))
}
df = pd.DataFrame(dic)
df

資料科學界，df 預期都會被當成一個DataFrame

改變df欄位名稱:

In [ ]:
rename_dic = {'col 1': 'x', 'col 2': '10x'}
df.rename(rename_dic, axis=1)

參數 axis，在pandas裡大部分函數預設處理的軸為row，以 axis=0 表示，而設為 1則代表以column套用該函示

也可用 df.columns 的方式改變欄位名稱

In [ ]:
df.columns = ['x(new)','10x(new)']+list(df.columns[2:])
df

使用pd.util.testing隨機建立DataFrame
-

In [ ]:
pd.util.testing.makeDataFrame().head(10)

head 函式預設用來顯示DataFrame前五筆數據，要顯示後面數據用tail

In [ ]:
pd.util.testing.makeDataFrame().head()

In [ ]:
pd.util.testing.makeDataFrame().tail()

也可用makeMixedDataFrame建立一個有各種資料型態的DataFrame

In [ ]:
pd.util.testing.makeMixedDataFrame()

將剪貼簿內容轉為DataFrame
-
可以從Excel，Google sheet或是網頁上複製表格來轉為DataFrame

In [ ]:
df = pd.read_clipboard()
df

考量重現性(reproducibility)，為了未來能重現當下結果，可以另存新檔

In [ ]:
df.to_csv("")

讀取線上CsV檔
-
只要有正確url及網路連線，就能將網路上CSV轉為DataFrame

範例為kaggle鐵達尼號競賽

In [ ]:
df = pd.read_csv('http://bit.ly/kaggletrain')
df.head()

In [ ]:
base_url = "https://data.taipei/api/getDatasetInfo/downloadResource?id={}&rid={}"
_id = "2f238b4f-1b27-4085-93e9-d684ef0e2735"
rid = "ea731a84-e4a1-4523-b981-b733beddbc1f"
csv_url = base_url.format(_id, rid)
df_raw = pd.read_csv(csv_url, encoding='big5')

# 複製一份做處理
df = df_raw.copy()

# 計算不同區不同性別的死亡、受傷人數
df['區序'] = df['區序'].apply(lambda x: ''.join([i for i in x if not i.isdigit()]))
df = (df[df['性別'].isin([1, 2])]
      .groupby(['區序', '性別'])[['死亡人數', '受傷人數']]
      .sum()
      .reset_index()
      .sort_values('受傷人數'))

df['性別'] = df['性別'].apply(lambda x: '男性' if x == 1 else '女性')
df = df.reset_index().drop('index', axis=1)

# 顯示結果
display(df_raw.head())
display(df.head())

優化記憶體使用量
-
可以用 df.info 查看df當前記憶體用量

In [ ]:
df.info(memory_usage='deep')

如果想讀入的DataFrame 很大，可以只讀入特定的欄位並將已知的分類型(categorical)欄位轉成category型態以節省記憶體 

In [ ]:
dtypes = {"Embarked": "category"}
cols = ['PassengerId', 'Name', 'Sex', 'Embarked']
df = pd.read_csv('http://bit.ly/kaggletrain', 
                 dtype=dtypes, usecols=cols)
df.info(memory_usage="deep")

透過減少讀入的欄位數並將object 轉換成 category 欄位，可以減少記憶體用量。

另外如果要取處理巨量CSV，也可用 chunksize參數來限制一次讀入的row

In [ ]:
from IPython.display import display

# chunksize=4 表示一次讀入 4 筆樣本
reader = pd.read_csv('http://bit.ly/kaggletrain', 
                     chunksize=4, usecols=cols)
# 秀出前兩個 chunks
for _, df_partial in zip(range(2), reader):
    display(df_partial)

讀入併合併多個CSV檔成單一DataFrame
-
使用 pd.concat 合併

reset_index 重置串接後的df索引

In [ ]:
from glob import glob
files = glob(r"C:\Users\jackiehuang\Desktop\python\pandas\passenger*.csv")

df = pd.concat([pd.read_csv(f) for f in files])
df.reset_index(drop=True)

前面有說Pandas 函式預設axis為0，代表以row為單位做特定操作，在pd.concat中依照row串接起來

如果是要把列資料串接，把axis設為1

In [ ]:
files = glob(r"C:\Users\jackiehuang\Desktop\python\pandas\feature_set*.csv")
pd.concat([pd.read_csv(f) for f in files], axis=1)

客製化DataFrame顯示設定
=
完整顯示所有欄位
-
有時df欄位太多，pandas會自動省略中間欄位

In [ ]:
df = pd.util.testing.makeCustomDataframe(5, 25)
df

但如果你一定要全部顯示，可用 pd.set_option 函式來改變 display.max_columns

In [ ]:
pd.set_option("display.max_columns", None)
df

也可用 T 來轉置(transpose)當前df

In [ ]:
# 注意轉置後 `head(15)` 代表選擇前 15 個欄位
df.T.head(15)

減少顯示欄位長度
-

In [ ]:
from IPython.display import display
print("display.max_colwidth 預設值：", 
      pd.get_option("display.max_colwidth"))

# 使用預設設定來顯示 DataFrame
df = pd.read_csv('http://bit.ly/kaggletrain')
display(df.head(3))

print("注意 Name 欄位的長度被改變了：")
# 客製化顯示（global）
pd.set_option("display.max_colwidth", 10)
df.head(3)

改變浮點數顯示位數
-

In [ ]:
pd.set_option("display.precision", 1)
df.head(3)


將設定初始化 (all好像有問題，之後再試試)

In [ ]:
#pd.reset_option('all')

執行 pd.describe_option() 可顯示所有可使用option

In [ ]:
pd.describe_option()

為特定DataFrame加樣式
-
不想像set_option 一樣改變到整個df，只要設定特定欄位

In [ ]:
# 隨機抽樣 10 筆數據來做 styling
df_sample = df.sample(n=10, random_state=9527).drop('Name', axis=1)
df_sample.Age.fillna(int(df.Age.mean()), inplace=True)
df_sample

使用 pandas Styler 底下的 format 函式來做到這件事情：

In [ ]:
import matplotlib
# 一個典型 chain pandas 函式的例子
(df_sample.style
     .format('{:.1f}', subset='Fare')
     .set_caption('★五顏六色の鐵達尼號數據集☆')
     .hide_index()
     .bar('Age', vmin=0)
     .highlight_max('Survived')
     .background_gradient('Greens', 
                          subset='Fare')
     .highlight_null()
)

- 將 Fare 欄位的數值顯示限制到小數後第一位
- 添加標題
- 隱藏索引
- 將 Age 欄位依數值大小畫條狀圖
- 將 Survived 最大的值 highlight
- 將 Fare 欄位依數值畫綠色的 colormap
- 將整個 DataFrame 的空值顯示為紅色

*實務上你應該思考什麼視覺變數是必要的，而不是盲目地添加樣式。

另外pandas 函式都會回傳處理後的結果，而不是直接修改原始 DataFrame。


數據清理 & 整理
=
處理空值
-

In [ ]:
df = pd.util.testing.makeMissingDataframe().head()
df

利用 fillna 函式將 DataFrame 裡頭所有不存在的值設為 0 或容易識別的值

In [ ]:
df.fillna(0) 

In [ ]:
df.fillna("Unknown")

捨棄不需要的行列
-

In [ ]:
df = pd.util.testing.makeDataFrame().head()
df

用 drop 函式來捨棄不需要的欄位

In [ ]:
columns = ['B', 'D']
df.drop(columns, axis=1)

In [ ]:
df.drop('KSWgxaQIUK')

重置並捨棄索引
-
很多時候你會想要重置一個 DataFrame 的索引，以方便使用 loc 或 iloc 屬性來存取想要的數據。

In [ ]:
df = pd.util.testing.makeDataFrame().head()
df

使用 reset_index 函式來重置此 DataFrame 

In [ ]:
df.reset_index(inplace=True)
df.iloc[:3, :]
# 豆知識：因為 iloc 是屬性而非函式，因此你得使用 [] 而非 () 存取數據

將函式的 inplace 參數設為 True 會讓 pandas 直接修改 df。一般來說 pandas 裡的函式並不會修改原始 DataFrame，這樣可以保證原始數據不會受到任何函式的影響。

當你不想要原來的 DataFrame df 受到 reset_index 函式的影響，則可以將處理後的結果交給一個新 DataFrame

In [ ]:
df = pd.util.testing.makeDataFrame().head()
df1 = df.reset_index(drop=True)
display(df)
display(df1)

In [ ]:
將字串切割成多個欄位
-

In [ ]:
df = pd.DataFrame({
    "name": ["大雄", "胖虎"], 
    "feature": ["膽小, 翻花繩", "粗魯, 演唱會"]
})
df

把這個 DataFrame 的 feature 欄位分成不同欄位，這時候利用 str 將字串取出，並透過 expand=True 將字串切割的結果擴大成（expand）成一個 DataFrame

In [ ]:
df[['性格', '特技']] = df.feature.str.split(',', expand=True)
df

注意我們使用 df[columns] = ... 的形式將字串切割出來的 2 個新欄位分別指定成 性格 與 特技。

將 list 分成多個欄位
-
有時候一個欄位裡頭的值為 Python list：

In [ ]:
df = pd.DataFrame({
    "name": ["大雄", "胖虎"], 
    "feature": [["膽小", "翻花繩"], ["粗魯", "演唱會"]]
})
df

使用 tolist 函式做到跟剛剛字串切割相同的效果：

In [ ]:
cols = ['性格', '特技']
pd.DataFrame(df.feature.tolist(), columns=cols)

也可以使用 apply(pd.Series)

In [ ]:
df.feature.apply(pd.Series)

取得想要關注的數據
=
基本數據切割
-

In [ ]:
df = pd.read_csv('http://bit.ly/kaggletrain')
df = df.drop("Name", axis=1)
df.head()

透過 loc 以及 :

In [ ]:
df.loc[:3, 'Pclass':'Ticket']

反向選取行列
-
透過 Python 常見的 [::-1] 語法，你可以輕易地改變 DataFrame 裡頭所有欄位的排列順序：

In [ ]:
df.loc[:3, ::-1]

In [ ]:
df.iloc[::-1, :5].head()

條件選取數據
-
在 pandas 裡頭最實用的選取技巧大概非遮罩（masking）莫屬了。遮罩讓 pandas 將符合特定條件的樣本回傳：

In [ ]:
male_and_age_over_70 = (df.Sex == 'male') & (df.Age > 70)
(df[male_and_age_over_70]
    .style
    .applymap(lambda x: 'background-color: rgb(153, 255, 51)', 
              subset=pd.IndexSlice[:, 'Sex':'Age']))
# 跟 df[(df.Sex == 'male') & (df.Age > 70)] 結果相同

存在多個判斷式時，有個準確說明遮罩意義的變數會讓你的程式碼好懂一點。

也可以使用 query 函式來達到跟遮罩一樣的效果：

In [ ]:
age = 70
df.query("Age > @age & Sex == 'male'")

可以使用 @ 來存取已經定義的 Python 變數 age 的值。

選擇任一欄有空值的樣本
-

In [ ]:
df[df.isnull().any(axis=1)].head() \
    .style.highlight_null()

選取或排除特定類型欄位
-
有時候你會想選取 DataFrame 裡特定數據類型（字串、數值、時間型態等）的欄位，這時你可以使用 select_dtypes 函式：

In [ ]:
df.select_dtypes(include='number').head()

可以用 include 選取特定類型欄位，也可以利用 exclude 參數來排除特定類型的欄位：

In [ ]:
# 建立一個有多種數據形態的 DataFrame
df_mix = pd.util.testing.makeMixedDataFrame()
display(df_mix)
display(df_mix.dtypes)
display(df_mix.select_dtypes(exclude=['datetime64', 'object']))

選取所有出現在 list 內的樣本
-
很多時候針對某一個特定欄位，你會想要取出所有出現在一個 list 的樣本。這時候你可以使用 isin 函式來做到這件事情：

In [ ]:
tickets = ["SC/Paris 2123", "PC 17475"]
df[df.Ticket.isin(tickets)]

選取某欄位為 top-k 值的樣本
-
很多時候你會想選取在某個欄位中前 k 大的所有樣本。這時你可以先利用 value_counts 函式找出該欄位前 k 多的值：

In [ ]:
top_k = 3
top_tickets = df.Ticket.value_counts()[:top_k]
top_tickets.index

也可以使用 pandas.Series 裡的 nlargest 函式取得相同結果：

In [ ]:
df.Ticket.value_counts().nlargest(top_k).index

接著利用上小節看過的 isin 函式就能輕鬆取得 Ticket欄位值為前 k 大值的樣本：

In [ ]:
df[df.Ticket.isin(top_tickets.index)].head()

找出符合特定字串的樣本
-

有時你會對一個字串欄位做正規表示式（regular expression），取出符合某個 pattern 的所有樣本。

這時你可以使用 str 底下的 contains 函式：

In [ ]:
df = pd.read_csv('http://bit.ly/kaggletrain')
df[df.Name.str.contains("Mr\.")].head(5)

使用正規表示式選取數據
-
有時候你會想要依照一些規則來選取 DataFrame 裡頭的值、索引或是欄位，尤其是在處理跟時間序列相關的數據：

In [ ]:
df_date = pd.util.testing.makeTimeDataFrame(freq='7D')
df_date.head(10)

假設你想將所有索引在 2000 年 2 月內的樣本取出，則可以透過 filter 函式達成這個目的：

In [ ]:
df_date.filter(regex="2000-02.*", axis=0)

選取從某時間點開始的區間樣本
-
在處理時間數據時，很多時候你會想要針對某個起始時間挑出前 t 個時間點的樣本。

In [ ]:
df_date.head(8)

在索引為時間型態的情況下，如果你想要把前 3 週的樣本取出，可以使用 first 函式：

In [ ]:
df_date.first('3W')

基本數據處理與轉換
=
對某一軸套用相同運算
-
你時常會需要對 DataFrame 裡頭的每一個欄位（縱軸）或是每一列（橫軸）做相同的運算。

比方說你想將鐵達尼號資料集內的 Survived 數值欄位轉換成人類容易理解的字串：

In [ ]:
# 重新讀取鐵達尼號數據
df_titanic = pd.read_csv('http://bit.ly/kaggletrain')
df_titanic = df_titanic.drop("Name", axis=1)

# 複製一份副本 DataFrame
df = df_titanic.copy()
columns = df.columns.tolist()[:4]

# 好戲登場
new_col = '存活'
columns.insert(1, new_col)  # 調整欄位順序用
df[new_col] = df.Survived.apply(lambda x: '倖存' if x else '死亡')
df.loc[:5, columns]

透過 apply 函式，我們把一個匿名函式 lambda 套用到整個 df.Survived Series 之上，並以此建立一個新的 存活 欄位。

對每一個樣本做自定義運算
-
想要把樣本（row）裡頭的多個欄位一次取出做運算並產生一個新的值。這時你可以自定義一個 Python function 並將 apply 函式套用到整個 DataFrame 之上：

In [ ]:
df = df_titanic.copy()
display(df.head())

# apply custom function 可以說是 pandas 裡最重要的技巧之一
d = {'male': '男性', 'female': '女性'} 
def generate_desc(row):
    return f"一名 {row['Age']} 歲的{d[row['Sex']]}"

df['描述'] = df.apply(generate_desc, axis=1)
df.loc[:4, 'Sex':]

將連續數值轉換成分類數據
-
可以使用 pd.cut 函式把一個連續數值（numerical）的欄位分成多個 groups 以方便對每個 groups 做統計。

pandas.cut(x, bins, right=True, labels=None, retbins=False, precision=3, include_lowest=False)

引數：

- x ： 必須是一維資料
- bins： 不同面元（不同範圍）型別:整數，序列如陣列, 和IntervalIndex
- right： 最後一個bins是否包含最右邊的資料，預設為True
- precision：精度 預設保留三位小數
- retbins： 即return bins 是否返回每一個bins的範圍 預設為False


In [ ]:
df = df_titanic.copy()

# 為了方便比較新舊欄位
columns = df.columns.tolist()
new_col = '年齡區間'
columns.insert(4, new_col)

# 將 numerical 轉換成 categorical 欄位
labels = [f'族群 {i}' for i in range(1, 11)]
df[new_col] = pd.cut(x=df.Age, 
                     bins=10, 
                     labels=labels)

# 可以排序切割後的 categorical 欄位
(df.sort_values(new_col, ascending=False)
   .reset_index()
   .loc[:5, columns]
)

如上所示，使用 pd.cut 函式建立出來的每個分類 族群 X 有大小之分，因此你可以輕易地使用 sort_values 函式排序樣本。

In [ ]:
df[new_col].dtype

將 DataFrame 隨機切成兩個子集
-
想將手上的 DataFrame 隨機切成兩個獨立的子集。選取其中一個子集來訓練機器學習模型是一個常見的情境。

要做到這件事情有很多種方法，你可以使用 scikit-learn 的 train_test_split 或是 numpy 的 np.random.randn，但假如你想要純 pandas 解法，可以使用 sample 函式：

In [ ]:
df_train = df_titanic.sample(frac=0.8, random_state=5566)
df_test = df_titanic.drop(df_train.index)

# 顯示結果，無特殊操作
display(df_train.head())
display(df_test.head())
print('各 DataFrame 大小：', 
      len(df_titanic), len(df_train), len(df_test))

這個解法的前提是原來的 DataFrame df_titanic 裡頭的索引是獨一無二的。另外記得設定 random_state 以方便別人重現你的結果。

用 SQL 的方式合併兩個 DataFrames
-
很多時候你會想要將兩個 DataFrames 依照某個共通的欄位（鍵值）合併成單一 DataFrame 以整合資訊。

In [ ]:
df_city = pd.DataFrame({
    'state': ['密蘇里州', '亞利桑那州', '肯塔基州', '紐約州'],
    'city': ['堪薩斯城', '鳳凰城', '路易維爾', '紐約市']
})
df_info = pd.DataFrame({
    'city': ['路易維爾', '堪薩斯城', '鳳凰城'],
    'population': [741096, 481420, 4039182],
    'feature': list('abc')})

display(df_city)
display(df_info)

DataFrame df_city 包含了幾個美國城市以及其對應的州名（state）；

DataFrame df_info 則包含城市名稱以及一些數據。如果你想將這兩個 DataFrames 合併（merge），可以使用非常方便的 merge 函式：

In [ ]:
pd.merge(left=df_city,
         right=df_info,
         how="left", # left outer join
         on="city", # 透過此欄位合併
         indicator=True # 顯示結果中每一列的來源
)

merge 函式運作方式就像 SQL 一樣，可以透過更改 how 參數來做：

-left：left outer join
-right：right outer join
-outer: full outer join
-inner： inner join

注意合併後的 DataFrame 的最後一列：因為是 left join，就算右側的 df_info 裡頭並沒有紐約市的資訊，我們也能把該城市保留在 merge 後的結果。

你還可以透過 indicator=True 的方式讓 pandas 幫我們新增一個 _merge 欄位，輕鬆了解紐約市只存在左側的 df_city 裡。

merge 函式強大之處在於能跟 SQL 一樣為我們抽象化如何合併兩個 DataFrames 的運算。

<br/>

存取並操作每一個樣本
-
我們前面看過，雖然一般可以直接使用 apply 函式來對每個樣本作運算，有時候你就是會想用 for 迴圈的方式把每個樣本取出處理。

這種時候你可以用 itertuples 函式：



In [ ]:
for row in df_city.itertuples(name='City'):
    print(f'{row.city}是{row.state}裡頭的一個城市')

itertuples 函式回傳的是 Python namedtuple

In [ ]:
from collections import namedtuple

City = namedtuple('City', ['Index', 'state', 'city'])
c = City(3, '紐約州', '紐約市')
c == row

簡單匯總 & 分析數據
=

取出某欄位 top k 的值
-
這與選取某欄位為 top-k 值的樣本小節一樣。

In [ ]:
df = df_titanic.copy()
display(df.head())
display(df.Ticket.value_counts().head(5).reset_index())

value_counts 函式預設就會把欄位裡頭的值依照出現頻率由高到低排序，因此搭配 head 函式就可以把最常出現的 top k 值選出。

<br/>

一行描述數值欄位
-
當你想要快速了解 DataFrame 裡所有數值欄位的統計數據（最小值、最大值、平均和中位數等）時可以使用 describe 函式：

In [ ]:
df.describe()

你也可以用取得想要關注的數據一節的技巧來選取自己關心的統計數據：

In [ ]:
df.describe().loc[['mean', 'std'], 'Survived':'Age']

找出欄位裡所有出現過的值
-
針對特定欄位使用 unique 函式即可：

In [ ]:
df.Sex.unique()

分組匯總結果
-
想要把 DataFrame 裡頭的樣本依照某些特性分門別類，並依此匯總各組（group）的統計數據。這種時候你可以用 groupby 函式。

In [ ]:
df = df_titanic.copy()
df.head()

依照它們的 Pclass 欄位值分組，並計算每組裡頭乘客們的平均年齡：

In [ ]:
df.groupby("Pclass").Age.mean()

也可以搭配 describe 函式來匯總各組的統計數據：

In [ ]:
df.groupby("Sex").Survived.describe()

也可以依照多個欄位分組，並利用 size 函式迅速地取得各組包含的樣本數：

In [ ]:
df.groupby(["Sex", 'Pclass']).size().unstack()

也可以用 agg 函式（aggregate，匯總）搭配 groupby 函式來將每一組樣本依照多種方式匯總：

In [ ]:
df.groupby(["Sex", 'Pclass']).Age.agg(['min', 'max', 'count'])

透過 unstack 函式能讓你產生跟 pivot_table 函式相同的結果：

In [ ]:
df.groupby(["Sex", 'Pclass']).Age.agg(['min', 'max', 'count']).unstack()

當然，你也可以直接使用 pivot_table 函式來匯總各組數據：

In [ ]:
df.pivot_table(index='Sex', 
               columns='Pclass', 
               values='Age', 
               aggfunc=['min', 'max', 'count'])

結合原始數據與匯總結果
-
不管是 groupby 搭配 agg 還是 pivot_table，匯總結果都會以另外一個全新的 DataFrame 表示。

有時候你會想直接把各組匯總的結果放到原本的 DataFrame 裡頭，方便比較原始樣本與匯總結果的差異。這時你可以使用 transform 函式：

In [ ]:
df = df_titanic.copy()
df['Avg_age'] = df.groupby("Sex").Age.transform("mean")
df['Above_avg_age'] = df.apply(lambda x: 'yes' if x.Age > x.Avg_age else 'no', 
                               axis=1)
# styling
(df.loc[:4, 'Sex':]
 .style
 .highlight_max(subset=['Avg_age'])
 .applymap(lambda x: 'background-color: rgb(153, 255, 51)', 
           subset=pd.IndexSlice[[0, 4], ['Age', 'Above_avg_age']])
)

對時間數據做匯總
-
給定一個跟時間相關的 DataFrame：

In [ ]:
df_date = pd.util.testing.makeTimeDataFrame(freq='Q').head(10) * 10
df_date

你可以利用 resample 函式來依照不同時間粒度匯總這個時間 DataFrame：

In [ ]:
(df_date.resample('Y').A.max()
 .reset_index()
 .rename({'index': 'year'}, axis=1)
)

此例中將不同年份（Year）的樣本分組，並從每一組的欄位 A 中選出最大值。

你可以查看官方 resampling 說明文件來了解還有什麼時間粒度可以選擇（分鐘、小時、月份等等）。

另外小細節是你可以利用 numpy 的 broadcasting 運算輕鬆地將 DataFrame 裡的所有數值做操作（初始 df_date 時的用到的 * 10）。

<br/>

簡易繪圖並修改預設樣式
-
在 Python 世界裡有很多數據視覺化工具供你選擇，比方說經典的 Matplotlib 以及在淺談神經機器翻譯一文中被我拿來視覺化矩陣運算的 Seaborn。

你也可以使用前面看過的 Chartify 搭配 pandas 畫出美麗圖表。

但有時，你只需要 pandas 內建的 plot 函式就能輕鬆地將一個 DataFrame 轉成統計圖：

In [ ]:
# 用 `plot` 一行畫圖。前兩行只是改變預設樣式
import matplotlib.pyplot as plt
plt.style.use("ggplot")
df.groupby("Pclass").Survived.count().plot(kind="barh");

我們都是視覺動物，pandas 的 plot 函式讓你在進行探索型數據分析（Exploratory Data Analysis, EDA）、試著快速了解手上數據集時十分方便。

另外 pandas 底層預設使用 Matplotlib 繪圖，而用過 Matplotlib 的人都知道其初始的繪圖樣式實在不太討喜。

你可以透過 plt.style.available 查看所有可供使用的繪圖樣式（style），並將喜歡的樣式透過 plt.style.use() 套用到所有 DataFrames 的 plot 函式：

In [ ]:
plt.style.available

與 pandas 相得益彰的實用工具
=

In [ ]:
df = df_titanic.copy()
df.head()

tqdm：了解你的數據處理進度
-
tqdm 是一個十分強大的 Python 進度條工具，且有整合 pandas。此工具可以幫助我們了解 DataFrame apply 函式的進度。回想一下我們在對某一軸套用相同運算一節做的一個簡單 apply 運算：

In [ ]:
df['存活'] = df.Survived.apply(lambda x: '倖存' if x else '死亡')
df.loc[:5, 'Survived':'存活']

在這個不到 1,000 筆的 DataFrame 做這樣的簡單運算不用一秒鐘，但實務上你可能常常需要對幾十萬、幾百萬筆數據分別做複雜的運算，這時候了解執行進度就是一件非常重要的事情。

In [ ]:
from tqdm import tqdm_notebook
tqdm_notebook().pandas()
#clear_output()

# 只需將 `apply` 替換成 `progress_apply`
df['存活'] = df.Survived.progress_apply(lambda x: '倖存' if x else '死亡')
df.loc[:5, 'Survived':'存活']

透過使用 progress_apply 函式，我們可以得到跟使用 apply 函式一樣的結果，附贈進度條。

在你 apply 的函式很複雜且樣本數很大時，你會很感謝有進度條的存在。你可以查看 tqdm repo 了解更多使用案例

<br/>

swifter：加速你的數據處理(windows好像沒有)
=
swifter 函式庫能以最有效率的方式執行 apply 函式。


In [ ]:
'''
import swifter
df = pd.DataFrame(pd.np.random.rand(1000000, 1), columns=['x'])

%timeit -n 10 df['x2'] = df['x'].apply(lambda x: x**2)
%timeit -n 10 df['x2'] = df['x'].swifter.apply(lambda x: x**2)
'''

qgrid：即時排序、篩選及編輯你的 DataFrame(安裝失敗)
-
qgrid 是一個能讓你在 Jupyter 筆記本裡使用簡單 UI 瀏覽並操作 DataFrame 的 widget。

In [ ]:
'''
import qgrid
qgrid.set_grid_option('maxVisibleRows', 7)
q = qgrid.show_grid(df_titanic)
q
'''

pandas-profiling：你的一鍵 EDA 神器
-
當碰到新的資料集時，你常會需要做 EDA 來探索並了解手上的數據。常見的基礎步驟有：

- 隨機抽樣 DataFrame 並用肉眼觀察樣本
- 了解樣本數有多少、各類型的變數有多少
- 了解 DataFrame 裡有多少空值
- 看數值欄位的分佈，欄位之間的相關係數
- 看分類型欄位的有多少不同的值，分佈為何

等等。這些在你熟悉本文的 pandas 技巧以後都應該跟吃飯喝水一樣，但很多時候就是例行公事。

這時你可以使用 pandas-profiling 讓它自動產生簡單報表，總結 DataFrame 裡的數據性質：

In [8]:
import pandas_profiling
df = df_titanic.copy()

# 一行報表：將想觀察的 DataFrame 丟進去就完工了
pandas_profiling.ProfileReport(df)

Render HTML: 100%|██████████| 1/1 [00:03<00:00,  4.00s/it]
